# Creating a Custom Container

This notebook builds a custom container that we can deploy to a SageMaker endpoint. It uses `docker` commands directly and therefore doesn't run from a SageMaker Studio notebook.

This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

In [8]:
import sagemaker
import boto3

region = boto3.Session().region_name

## Step 1 - Creating an ECR Repository

We want to create an ECR repository to host the image of our custom endpoint.

In [14]:
REPOSITORY_NAME = "penguins"

!aws ecr create-repository --repository-name $REPOSITORY_NAME

repository = !aws ecr describe-repositories \
    --repository-names $REPOSITORY_NAME \
    --query "repositories[0].repositoryUri"

repository_uri = repository[0][1:-1]
repository = repository_uri[0:repository_uri.index("/")]

repository_uri


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'penguins' already exists in the registry with id '325223348818'


'325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins'

## Step 2 - Build and Push the Image to ECR

Now we can build the image and push it to Elastic Container Registry.

In [15]:
# The custom image we are creating inherits from a built-in SageMaker image. 
# To get access to it, we need to authenticate with their ECR repository.
!aws ecr get-login-password --region $region | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

# Build the image using the Dockerfile located in the current directory.
!docker build -t $repository_uri:latest .

# To push the image we need to authenticate with our ECR repository.
!aws ecr get-login-password | docker login --username AWS --password-stdin $repository

# Push the image to the repository
!docker push $repository_uri:latest

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  133.1kB
Step 1/10 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4-cpu AS sagemaker
 ---> f0835ba9191e
Step 2/10 : RUN apt-get clean &&     apt-get update -y &&     apt-get install -y --no-install-recommends     libgl1-mesa-glx
 ---> Using cache
 ---> 1384d0f73d12
Step 3/10 : RUN pip install --upgrade pip
 ---> Using cache
 ---> ed101bf7a6f2
Step 4/10 : RUN pip install pandas
 ---> Using cache
 ---> 34715e9b9c85
Step 5/10 : RUN pip install numpy
 ---> Using cache
 ---> 2b3dc8e6ef98
Step 6/10 : RUN pip install requests
 ---> Using cache
 ---> 4024b78363a0
Step 7/10 : RUN pip install tensorflow==2.4
 ---> Using cache
 ---> ba3739fd7725
Step 8/10 : RUN pip install scikit-learn==0.23.2